### Настройка окружения

Для начала вам необходимо выполнить ряд команд чтобы настроить окружение для дальнейшей работы, это позволит первое время не заниматься долгим деплоем, а сразу начать писать код и работать с airflow.

In [ ]:
# Установка Airflow
!pip install apache-airflow==2.1.4

!airflow db init

In [ ]:
# Создадим папку dags
# В этой папке лежат скрипты для создания дагов
# Это стандартное имя для  данной папки
!mkdir /root/airflow/dags
!touch /root/airflow/dags/dag.py

mkdir: cannot create directory ‘/root/airflow/dags’: File exists


In [ ]:
# Последующие команды не имеют отношения к Airflow
# Они нужни только для корректоной работы веб приложения
# в среде Google Colab

!pip install pyngrok

!ngrok authtoken '' # найти его можно https://dashboard.ngrok.com/get-started/setup 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
!airflow users create \
          --username admin \
          --firstname admin \
          --lastname admin \
          --role Admin \
          --email admin@example.org \
          -p 12345

/usr/local/lib/python3.9/dist-packages/flask_appbuilder/models/sqla/interface.py:68 SAWarning: relationship 'DagRun.serialized_dag' will copy column serialized_dag.dag_id to column dag_run.dag_id, which conflicts with relationship(s): 'DagRun.task_instances' (copies task_instance.dag_id to dag_run.dag_id), 'TaskInstance.dag_run' (copies task_instance.dag_id to dag_run.dag_id). If this is not the intention, consider if these relationships should be linked with back_populates, or if viewonly=True should be applied to one or more if they are read-only. For the less common case that foreign key constraints are partially overlapping, the orm.foreign() annotation can be used to isolate the columns that should be written towards.   To silence this warning, add the parameter 'overlaps="dag_run,task_instances"' to the 'DagRun.serialized_dag' relationship. (Background on this error at: https://sqlalche.me/e/14/qzyx)
/usr/local/lib/python3.9/dist-packages/flask_appbuilder/models/sqla/interface.py

In [ ]:
# Включим веб-сервер
!airflow webserver -p 18273 -D

/usr/local/lib/python3.9/dist-packages/flask_appbuilder/models/sqla/interface.py:68 SAWarning: relationship 'DagRun.serialized_dag' will copy column serialized_dag.dag_id to column dag_run.dag_id, which conflicts with relationship(s): 'DagRun.task_instances' (copies task_instance.dag_id to dag_run.dag_id), 'TaskInstance.dag_run' (copies task_instance.dag_id to dag_run.dag_id). If this is not the intention, consider if these relationships should be linked with back_populates, or if viewonly=True should be applied to one or more if they are read-only. For the less common case that foreign key constraints are partially overlapping, the orm.foreign() annotation can be used to isolate the columns that should be written towards.   To silence this warning, add the parameter 'overlaps="dag_run,task_instances"' to the 'DagRun.serialized_dag' relationship. (Background on this error at: https://sqlalche.me/e/14/qzyx)
/usr/local/lib/python3.9/dist-packages/flask_appbuilder/models/sqla/interface.py

In [ ]:
# Запуск шедулера
!airflow scheduler -D

/usr/local/lib/python3.9/dist-packages/airflow/utils/cli.py:149 SAWarning: relationship 'DagRun.serialized_dag' will copy column serialized_dag.dag_id to column dag_run.dag_id, which conflicts with relationship(s): 'DagRun.task_instances' (copies task_instance.dag_id to dag_run.dag_id), 'TaskInstance.dag_run' (copies task_instance.dag_id to dag_run.dag_id). If this is not the intention, consider if these relationships should be linked with back_populates, or if viewonly=True should be applied to one or more if they are read-only. For the less common case that foreign key constraints are partially overlapping, the orm.foreign() annotation can be used to isolate the columns that should be written towards.   To silence this warning, add the parameter 'overlaps="dag_run,task_instances"' to the 'DagRun.serialized_dag' relationship. (Background on this error at: https://sqlalche.me/e/14/qzyx)
/usr/local/lib/python3.9/dist-packages/airflow/utils/cli.py:149 SAWarning: relationship 'SerializedD

In [ ]:
# Эта команда просто отображет веб морду на другой адрес
# Его вы можете найти https://dashboard.ngrok.com/endpoints/status
# При каждом отключении ссылка будет меняться
!nohup ngrok http -log=stdout 18273 > /dev/null &

nohup: redirecting stderr to stdout


### Задание на понимание контекста задачи

Вставьте необходимый код в ячейки

```python
from airflow import DAG
from airflow.operators.dummy_operator import DummyOperator
from airflow.operators.python_operator import PythonOperator
from datetime import datetime


def my_func(hello, date, **context):
  print(hello)
  print(date)
  print(<ВАШ КОД: Обратиться  через контекст к имени задачи>)


with DAG('dag', schedule_interval='@daily', 
          start_date=<ВАШ КОД: 2021-01-01>,
          end_date=<ВАШ КОД: 2021-01-10>) as dag:

  python_task	= PythonOperator(
    task_id='python_task', 
    python_callable=my_func,
    op_kwargs= {
      'hello': <ВАШ КОД: передать строку Hello World>,
      'date': <ВАШ КОД: передать сюда дату запуска, через макрос>
      }
    )
```



In [ ]:
## ВАШЕ РЕШЕНИЕ
from airflow import DAG
from airflow.operators.dummy_operator import DummyOperator
from airflow.operators.python_operator import PythonOperator
from datetime import datetime


def my_func(hello, date, **context):
  print(hello)
  print(date)
  print(context['task'])


with DAG('dag', schedule_interval='@daily', 
          start_date= datetime(2021,1,1),
          end_date=datetime(2021,1,10)) as dag:

  python_task    = PythonOperator(
    task_id='python_task', 
    python_callable=my_func,
    op_kwargs= {
      'hello': 'Hello World',
      'date':  '{{ execution_date }}'
      }
    )